In [3]:
import numpy as np
from scipy.interpolate import griddata
from scipy.spatial import cKDTree
import laspy
from scipy import ndimage
from tqdm import tqdm
import os

In [ ]:
def print_las_info(las_path):
    """
    Выводит основную информацию о LAS/LAZ файле:
    количество точек, границы (X, Y, Z), размер области и примерную плотность.
    x, y - 
    z - высота
    """
    with laspy.open(las_path) as f:
        header = f.header
        num_points = header.point_count
        min_x, max_x = header.mins[0], header.maxs[0]
        min_y, max_y = header.mins[1], header.maxs[1]
        min_z, max_z = header.mins[2], header.maxs[2]

        width = max_x - min_x
        lenght = max_y - min_y
        area = width * lenght if width > 0 and lenght > 0 else np.nan
        density = num_points / area if area and not np.isnan(area) else np.nan

        print(f"\nФайл: {os.path.basename(las_path)}")
        print(f"Формат версии: {header.version}")
        print(f"Количество точек: {num_points:,}")
        print(f"Диапазон координат (X, Y, Z):")
        print(f"  X: {min_x:.2f} - {max_x:.2f}  (ширина {width:.2f} м)")
        print(f"  Y: {min_y:.2f} - {max_y:.2f}  (размах {lenght:.2f} м)")
        print(f"  Z: {min_z:.2f} - {max_z:.2f}  (высота {max_z - min_z:.2f} м)")
        print(f"Площадь покрытия: {area/1e6:.2f} км²")

        print(f"Средняя плотность: {density:.2f} точек/м²\n")

        # Первые несколько точек
        print("Пример точек:")
        for points in f.chunk_iterator(3):
            for i in range(3):
                print(f"x: {points.x[i]:.2f} y: {points.y[i]:.2f} z: {points.z[i]:.2f}")
            break

In [ ]:
def progressive_morphological_filter(points, cell_size=1.0, max_window_size=20, 
                                   initial_threshold=0.5, slope=0.3):
    """
    PMF filter, returns mask (True = ground)
    """
    min_x, min_y = np.min(points[:, :2], axis=0)
    max_x, max_y = np.max(points[:, :2], axis=0)
    
    cols = int(np.ceil((max_x - min_x) / cell_size))
    rows = int(np.ceil((max_y - min_y) / cell_size))
    
    grid = np.full((rows, cols), np.inf)
    for x, y, z in points:
        col = int((x - min_x) / cell_size)
        row = int((y - min_y) / cell_size)
        if 0 <= row < rows and 0 <= col < cols:
            if z < grid[row, col]:
                grid[row, col] = z
    
    ground_mask = np.zeros(len(points), dtype=bool)
    
    window_sizes = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
    
    for window_size in window_sizes:
        if window_size > max_window_size:
            break
            
        footprint = np.ones((window_size, window_size))
        eroded = ndimage.grey_erosion(grid, footprint=footprint)
        opened = ndimage.grey_dilation(eroded, footprint=footprint)
        
        current_threshold = initial_threshold + slope * (window_size / 2) * cell_size
        
        for i, (x, y, z) in enumerate(points):
            col = int((x - min_x) / cell_size)
            row = int((y - min_y) / cell_size)
            
            if 0 <= row < rows and 0 <= col < cols:
                if z - opened[row, col] <= current_threshold:
                    ground_mask[i] = True
    
    return ground_mask

In [ ]:
def save_las_points(
    original_las: laspy.LasData,
    points: np.ndarray,
    output_path: str,
    copy_attributes: bool = True
):
    """
    Сохраняет LAS-файл из массива точек, копируя метаданные и при желании атрибуты.

    Args:
        original_las (laspy.LasData): исходный LAS-файл (для копирования header и метаданных)
        points (np.ndarray): Nx3 массив координат (x, y, z)
        output_path (str): путь для сохранения
        copy_attributes (bool): копировать ли intensity, classification и т.д.
    """
    
    # создаём новый заголовок на основе оригинала
    header = laspy.LasHeader(
        point_format=original_las.header.point_format,
        version=original_las.header.version
    )
    header.scales = original_las.header.scales
    header.offsets = original_las.header.offsets

    # создаём новый LAS-объект
    new_las = laspy.LasData(header)

    # записываем координаты
    new_las.x = points[:, 0]
    new_las.y = points[:, 1]
    new_las.z = points[:, 2]

    # при необходимости копируем остальные поля
    if copy_attributes:
        attrs = [
            'intensity', 'return_number', 'number_of_returns',
            'classification', 'user_data', 'gps_time',
            'red', 'green', 'blue'
        ]
        for attr in attrs:
            if hasattr(original_las, attr):
                data = getattr(original_las, attr)
                if len(data) == len(original_las.points):
                    new_las[attr] = data[:len(points)]  # если точек меньше, копируем соответствующее количество

    # сохраняем
    new_las.write(output_path)
    print(f"Saved {len(points):,} points to {output_path}")


# Отделение земли с помощью PMF

In [ ]:
input_las_path = "data/test_scan.las"

las = laspy.read(input_las_path)
points = np.vstack((las.x, las.y, las.z)).transpose()
# points = [[x1, y1, z1],
#           [x2, y2, z2]...]


print("before applied filter")
ground_mask = progressive_morphological_filter(points, 
                                               cell_size=0.5,
                                               max_window_size=15,
                                               initial_threshold=0.3,
                                               slope=0.2)


# Invert ground
non_ground_mask = ~ground_mask

print(f"Non ground points: {np.sum(non_ground_mask)}")
print(f"Ground points: {np.sum(ground_mask)}")

ground_points     = points[ground_mask]
non_ground_points = points[non_ground_mask]


save_las_points(las, ground_points, "ground.las")
save_las_points(las, non_ground_points, "non_ground.las")

# Фильтрация точек земли

In [ ]:
max_ground_height = 2
ground_mask_fix = ground_points[:, 2] <= max_ground_height

print(f"Ground points over threshold: {np.sum(~ground_mask_fix)}")

ground_points_v2 = ground_points[ground_mask_fix]

save_las_points(las, ground_points_v2, "ground_v2.las")

# Нормализация 

In [ ]:
def normalize_point_cloud(points, ground_points, method='linear'):
    """
    Нормализует высоту облака относительно поверхности земли.

    Args:
        points (np.ndarray): Nx3 массив (x, y, z) — точки, которые хотим нормализовать.
        ground_points (np.ndarray): Mx3 массив (x, y, z) — точки земли.
        method (str): метод интерполяции ('linear', 'nearest', 'cubic').

    Returns:
        np.ndarray: массив таких же точек, но с нормализованным z.
    """
    print("⚙️ Интерполяция поверхности земли...")

    # интерполяция z_ground по координатам x, y
    ground_z = griddata(
        ground_points[:, :2],
        ground_points[:, 2],
        points[:, :2],
        method=method,
        fill_value=np.nan
    )

    # нормализуем
    normalized_points = points.copy()
    normalized_points[:, 2] = points[:, 2] - ground_z

    # удаляем точки с NaN (за пределами модели)
    mask_valid = ~np.isnan(normalized_points[:, 2])
    normalized_points = normalized_points[mask_valid]

    # Убираем точки ниже чем минимальная точка земли
    # treshold = min(ground_z)
    treshold = 0
    lower_than_ground = normalized_points[:, 2] >= treshold
    normalized_points = normalized_points[lower_than_ground]
    

    print(f"✅ Нормализовано {len(normalized_points):,} точек (из {len(points):,})")

    return normalized_points


In [ ]:
normalized_points = normalize_point_cloud(non_ground_points, ground_points)
save_las_points(las, normalized_points, "normalized.las")

# Информация по полученым файлам

In [ ]:
print_las_info(input_las_path)

In [ ]:
print_las_info("ground.las")

In [ ]:
print_las_info("non_ground.las")


Файл: non_ground.las
Формат версии: 1.2
Количество точек: 1,182,178
Диапазон координат (X, Y, Z):
  X: -0.15 - 39.85  (ширина 40.00 м)
  Y: -61.56 - -27.03  (размах 34.53 м)
  Z: -3.09 - 21.20  (высота 24.29 м)
Площадь покрытия: 0.00 км²
Средняя плотность: 855.93 точек/м²

Пример точек:
x: -0.01 y: -52.14 z: -0.91
x: -0.01 y: -52.15 z: -0.89
x: -0.02 y: -27.03 z: -3.09


In [ ]:
print_las_info("ground_v2.las")

In [ ]:
print_las_info("normalized.las")